# IMDB Random Forests Regression
![ImdbIcon](../images/imdbheader.jpg)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report

C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
revenue_df = pd.read_csv('../data/revenue_df.csv')

In [3]:
revenue_df.columns

Index(['movie_title', 'year', 'tagline', 'plot', 'cast', 'duration', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama',
       'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 'avg_vote',
       'total_votes', 'us_voters_votes', 'votes', 'votes_1', 'votes_2',
       'votes_3', 'votes_4', 'votes_5', 'votes_6', 'votes_7', 'votes_8',
       'votes_9', 'votes_10', 'popularity', 'director_score', 'actor_score',
       'actress_score', 'tagline_sentiment', 'plot_sentiment', 'total_score',
       'profitable', 'budget', 'revenue', 'budget_adj', 'revenue_adj'],
      dtype='object')

## Random Forests Regression

In [4]:
features = ['duration', 'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 
            'Crime', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 
            'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 'avg_vote',
            'total_votes', 'us_voters_votes', 'votes', 'votes_1', 'votes_2', 'votes_3', 'votes_4', 
            'votes_5', 'votes_6', 'votes_7', 'votes_8', 'votes_9', 'votes_10', 'popularity', 
            'director_score', 'actor_score', 'actress_score', 'tagline_sentiment', 'plot_sentiment', 
            'total_score', 'budget']

In [5]:
X = revenue_df[features]
y = revenue_df['revenue']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = 42)

In [7]:
rf = RandomForestRegressor()

In [8]:
print("Cross Val Score:", cross_val_score(rf, X_train, y_train, cv = 5).mean())

Cross Val Score: 0.6654816047422895


In [9]:
model_params = {}
count = 0

In [14]:
rf_params = {
    'max_depth': [9, 10, 11],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 3, 4],
    'n_estimators': [350, 400, 450],
}

gs = GridSearchCV(rf, param_grid=rf_params, cv = 5, n_jobs = 12)

gs.fit(X_train, y_train)

count += 1

gs.best_params_['best_score'] = gs.best_score_

gs.best_params_['training_score'] = gs.score(X_train, y_train)

gs.best_params_['testing_score'] = gs.score(X_test, y_test)

model_params[f'model_{count}'] = gs.best_params_

model_df = pd.DataFrame.from_dict(model_params, orient='index')

model_df

,max_depth,min_samples_leaf,min_samples_split,n_estimators,best_score,training_score,testing_score
model_1,3,1,2,100,0.631316,0.756501,0.468206
model_2,5,2,3,100,0.662146,0.835330,0.504975
model_3,7,1,4,300,0.667750,0.897355,0.535553
model_4,9,1,3,400,0.671402,0.928145,0.533396
model_5,11,1,2,350,0.672237,0.947054,0.545173


## AdaBoostRegressor

In [18]:
model_params_two = {}
count_two = 0

In [22]:
ada = AdaBoostRegressor(base_estimator=RandomForestRegressor())

ada_params = {
    'base_estimator__max_depth': [3, 4, 5],
    'learning_rate': [.80, .85, .90],
    'n_estimators': [300, 400, 500],
}

gs_two = GridSearchCV(ada, param_grid=ada_params, cv=3, n_jobs = 12)

gs_two.fit(X_train, y_train)

count_two += 1

gs_two.best_params_['best_score'] = gs_two.best_score_

gs_two.best_params_['training_score'] = gs_two.score(X_train, y_train)

gs_two.best_params_['testing_score'] = gs_two.score(X_test, y_test)

model_params_two[f'model_{count_two}'] = gs_two.best_params_

model_df_two = pd.DataFrame.from_dict(model_params_two, orient='index')

model_df_two

,base_estimator__max_depth,learning_rate,n_estimators,best_score,training_score,testing_score
model_1,3,1.00,300,0.600364,0.768746,0.427752
model_2,4,0.95,300,0.619965,0.833536,0.490926
model_3,4,0.90,400,0.640080,0.842121,0.455173
model_4,4,0.80,500,0.644468,0.851919,0.468607
